In [18]:
%load_ext autoreload
%autoreload 2
import os
import matplotlib.pyplot as plt
import seaborn as sns
from os.path import join
from tqdm import tqdm
import pandas as pd
import pickle as pkl
import notebook_helper
import sys
import imodelsx
import imodelsx.process_results
import mprompt.data.data
from mprompt.data.data import TASKS_D3
TASK_NAMES = list(TASKS_D3.keys())
sys.path.append('../experiments/')
results_dir = '/home/chansingh/mntv1/mprompt/feb18_synthetic_sweep'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
r = imodelsx.process_results.get_results_df(results_dir, use_cached=False)
r = imodelsx.process_results.fill_missing_args_with_default(r, experiment_filename='01_explain.py')
r['top_contains_keywords'] = r['score_contains_keywords'].apply(lambda x: x[0])
r['task_str'] = r.apply(lambda row: mprompt.data.data.get_task_str(row['module_name'], row['module_num']), axis=1)
r['task_keyword'] = r['task_str'].apply(lambda task_str: mprompt.data.data.get_task_keyword(task_str))
r['task_name (groundtruth)'] = r['task_str'].apply(lambda s: s.split('_')[-1])
r = r.sort_values(by='top_score_synthetic', ascending=False).round(3)

100%|██████████| 43/43 [00:03<00:00, 12.77it/s]


In [25]:
default_params = {
    'noise_ngram_scores': 0,
    'module_num_restrict': -1,
}
rd = r
for k, v in default_params.items():
    rd = rd[rd[k] == v]

# Top results

In [26]:
with pd.option_context('display.max_colwidth', -1, 'display.max_rows', 200):
    display(rd[['task_keyword', 'top_explanation_init_strs', 'top_contains_keywords',
   'top_score_synthetic', 'module_num']])

,task_keyword,top_explanation_init_strs,top_contains_keywords,top_score_synthetic,module_num
21,Hillary,"name ""hillary""",True,0.656,17
3,Hillary,"name ""hillary""",True,0.656,16
33,pro-life,discussion of abortion and the related topics of pro-life and pro-choice,True,0.399,20
2,politics,politics,True,0.350,53
18,evacuation,"concept of evacuation, which is the process of leaving a place or area due to danger or emergency",False,0.295,5
14,toxic,idea of danger and toxicity,True,0.295,19
15,grammatical,linguistics and language,False,0.282,27
42,religious,religion,True,0.266,3
38,atheistic,"religion and belief systems, specifically atheism",True,0.254,4
6,sexist,sexism and feminism,True,0.243,28


### Accuracy results

In [23]:
acc = r['top_contains_keywords'].mean()
print('acc', acc)

acc 0.6341463414634146


# Details

In [11]:
# display full strings
with pd.option_context('display.max_colwidth', -1, 'display.max_rows', 200):
    display(
        r
        .filter(['task_str', 'top_explanation_init_strs', 'top_contains_keywords', 'top_score_synthetic', 'module_num'])
        # .filter(['top_score_contains_keywords', 'top_score_synthetic', 'top_explanation_init_strs', 'explanation_init_ngrams', 'module_num'])
        # .filter(['top_explanation_init_strs', 'explanation_init_ngrams', 'top_strs_added'])
        # .filter(['explanation_init_strs', 'explanation_init_ngrams',])
        # .head(n=20)
    )

,task_str,top_explanation_init_strs,top_contains_keywords,top_score_synthetic,module_num
8,d3_17_hillary,"name ""hillary""",True,0.656,17
2,d3_16_hillary,"name ""hillary""",True,0.656,16
14,d3_20_pro-life,discussion of abortion and the related topics of pro-life and pro-choice,True,0.399,20
1,d3_53_politic,politics,True,0.350,53
18,d3_3_god,religion,True,0.266,3
17,d3_4_atheism,"religion and belief systems, specifically atheism",True,0.254,4
0,d3_23_computer science,"""computer""",True,0.231,23
11,d3_29_sexis,feminism,True,0.231,29
15,d3_7_crime,crime and criminal activity,True,0.214,7
7,d3_13_water,water,True,0.213,13
